In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as image
import glob 
import os

In [2]:
images_fp='./images'

In [3]:
image_names=[os.path.basename(file) for file in glob.glob(os.path.join(images_fp,'*.jpg')) ]

In [4]:
len(image_names)

7390

In [5]:
labels=[name.split('_')[:-1:] for name in image_names]

In [6]:
labels

[['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'],
 ['Abyssinian'

In [7]:
def label_encode(label):
    if label=='Abyssinian':return 0 
    elif label=='american bulldog':return 1
    elif label=='american pit bull terrier':return 2 
    elif label=='basset hound':return 3 
    elif label=='beagle':return 4 
    elif label=='Bengal':return 5 
    elif label=='British Shorthair':return 6 
    elif label=='Egyptian Mau':return 7 
    elif label=='boxer':return 8 
    elif label=='chihuahua':return 9 
    elif label=='english cocker spaniel':return 10 
    elif label=='english setter':return 11
    elif label=='german shorthaired':return 12
    elif label=='great pyrenees':return 13
    elif label=='Birman':return 14
    elif label=='Bombay':return 15
    

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
features=[]
labels=[]
IMAGE_SIZE =(224,224)

for name in image_names:
    label=' '.join(name.split('_')[:-1:])
    label_encoded=label_encode(label)
    if label_encoded != None:
        img =load_img(os.path.join(images_fp,name))
        img=tf.image.resize_with_pad(img_to_array(img,dtype='uint8'),*IMAGE_SIZE).numpy().astype('uint8')
        image=np.array(img)
        features.append(image)
        labels.append(label_encoded)

In [ ]:
features

In [ ]:
labels

In [ ]:
f_ary=np.array(features)
l_ary=np.array(labels)

In [ ]:
labels_one_hot=pd.get_dummies(l_ary)

In [ ]:
labels_one_hot

In [ ]:
# !pip install scikit-learn
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(f_ary,labels_one_hot,test_size=0.2,random_state=42)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.25,random_state=1)

In [ ]:
from tensorflow.keras import layers,Input,Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as pp
from tensorflow.keras.layers import RandomFlip,RandomRotation,Dense,Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
data_augmentation = Sequential([RandomFlip("horizontal_and_vertical"),RandomRotation(0.2)])
prediction_layers=Dense(16,activation='softmax')

In [ ]:
resnet_model = ResNet50(include_top=False,pooling='avg',weights='imagenet')
resnet_model.trainable=False

In [ ]:
preprocess_input=pp

In [ ]:
inputs=Input(shape=(224,224,3))
X=data_augmentation(inputs)
X=preprocess_input(X)
X=resnet_model(X,training=False)
X=Dropout(0.2)(X)
outputs = prediction_layers(X)
model=Model(inputs,outputs)


In [ ]:
model.compile(optimizer=Adam(),loss=CategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
 model_history = model.fit(x=x_train,y=y_train,validation_data=(x_val,y_val),epochs=20)

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

In [ ]:
epochs_range = range(20)
plt.figure(figsize=(15,8))

plt.subplot(1,2,1)
plt.plot(epochs_range,acc,label='Training accuracy')
plt.plot(epochs_range,val_acc,label='validition accuracy')
plt.legend(loc='upper right')
plt.title('training and validation accuracy')
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label='Training accuracy')
plt.plot(epochs_range,val_loss,label='validity accuracy')
plt.legend(loc='upper right')
plt.title('training and validation loss')

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred